### Load the model and make prediction
##### Input Args:

doc_dir = directory where the file with comments to be predicted

doc2vec_model_name = doc2vec model already trained

trained_clf = Trained classifier


In [2]:
from gensim.models import Doc2Vec
import re
import os
import random
from sklearn import metrics
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
def extract_words(sent):
    #print("line tokenizing : \n{0}".format(sent))
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent) #strip html tags
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent) # remoce apostrophes
    sent = re.sub(r'\W',' ', sent) # remove punctuation
    sent = re.sub(r'\s+', ' ', sent) # remove repeated space
    sent = sent.strip()
    return sent.split()

In [5]:
#Load doc to vec model
doc2vec_model_name = 'reviews.d2v'
doc2vec_model = Doc2Vec.load(doc2vec_model_name)

In [6]:
#Test the trained doc2vec model
cosine_similarity(
[doc2vec_model.infer_vector(extract_words("This is very bad video. I don't like it"))],
[doc2vec_model.infer_vector(extract_words("video sucks."))])

array([[0.73956066]], dtype=float32)

In [8]:
#Converts the comments to vector using doc2vec model trianed earlier
def get_doc2vec(model, doc_dir,comment_col_name='Comment'):
    comments_dfs = []
    
    for fname in sorted(os.listdir(doc_dir)):
        fname_full = doc_dir + "/" + fname
        print("file being read : {0}".format(fname_full))
        data = pd.read_csv(fname_full, sep='\t', header='infer')
        data['FNAME'] = fname
        data['WORDS'] = data[comment_col_name].map(lambda x: extract_words(x))
        data['SENTVECS'] = data['WORDS'].map(lambda x: model.infer_vector(x, steps=10))
        comments_dfs.append(data)

    comments = pd.concat(comments_dfs)
    return comments

In [9]:
doc_dir = "../indata"
comments = get_doc2vec(doc2vec_model, doc_dir)

files being read : ../indata/samsung-vr.csv
files being read : ../indata/tswift.csv


In [10]:
comments.head(5)

,Comment,CreateTimeStamp,Type,videoID,FNAME,WORDS,SENTVECS
0,I ll just leave this right here,2017-07-27T19:00:40.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,"[i, ll, just, leave, this, right, here]","[0.1917683, 0.15611713, 0.3199824, 0.051351413..."
1,How does sound work,2018-11-30T13:18:40.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,"[how, does, sound, work]","[-0.095913, 0.2271512, 0.16931242, -0.20964487..."
2,hey,2018-11-29T04:51:42.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,[hey],"[-0.0030196828, 0.13162734, 0.047134276, -0.08..."
3,Whats the song in the beginning,2018-11-23T20:46:13.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,"[whats, the, song, in, the, beginning]","[0.4138912, 0.23341818, 0.054907303, -0.089427..."
4,Can t compete with a PS VR,2018-11-21T10:54:45.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,"[can, t, compete, with, a, ps, vr]","[-0.080483355, 0.07329093, 0.14160651, 0.01256..."


In [11]:
print("No of files : {0}".format(comments.FNAME.unique()))
print("No of Type : {}".format(comments.Type.unique()))

No of files : ['samsung-vr.csv' 'tswift.csv']
No of Type : ['Comment' 'Reply']


In [12]:
trained_clf = 'doc2vec_rf.model'
loaded_clf = joblib.load(trained_clf)

/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [13]:
comments['SENTIMENT_RF'] = loaded_clf.predict(comments['SENTVECS'].tolist())

In [14]:
comments.head(5)

,Comment,CreateTimeStamp,Type,videoID,FNAME,WORDS,SENTVECS,SENTIMENT_RF
0,I ll just leave this right here,2017-07-27T19:00:40.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,"[i, ll, just, leave, this, right, here]","[0.1917683, 0.15611713, 0.3199824, 0.051351413...",1
1,How does sound work,2018-11-30T13:18:40.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,"[how, does, sound, work]","[-0.095913, 0.2271512, 0.16931242, -0.20964487...",0
2,hey,2018-11-29T04:51:42.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,[hey],"[-0.0030196828, 0.13162734, 0.047134276, -0.08...",0
3,Whats the song in the beginning,2018-11-23T20:46:13.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,"[whats, the, song, in, the, beginning]","[0.4138912, 0.23341818, 0.054907303, -0.089427...",0
4,Can t compete with a PS VR,2018-11-21T10:54:45.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,"[can, t, compete, with, a, ps, vr]","[-0.080483355, 0.07329093, 0.14160651, 0.01256...",1


In [15]:
trained_clf = 'doc2vec_kn.model'
loaded_clf = joblib.load(trained_clf)

In [16]:
comments['SENTIMENT_KN'] = loaded_clf.predict(comments['SENTVECS'].tolist())

In [19]:
comments.head(5)

,Comment,CreateTimeStamp,Type,videoID,FNAME,WORDS,SENTVECS,SENTIMENT_RF,SENTIMENT_KN
0,I ll just leave this right here,2017-07-27T19:00:40.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,"[i, ll, just, leave, this, right, here]","[0.1917683, 0.15611713, 0.3199824, 0.051351413...",1,0
1,How does sound work,2018-11-30T13:18:40.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,"[how, does, sound, work]","[-0.095913, 0.2271512, 0.16931242, -0.20964487...",0,0
2,hey,2018-11-29T04:51:42.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,[hey],"[-0.0030196828, 0.13162734, 0.047134276, -0.08...",0,0
3,Whats the song in the beginning,2018-11-23T20:46:13.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,"[whats, the, song, in, the, beginning]","[0.4138912, 0.23341818, 0.054907303, -0.089427...",0,1
4,Can t compete with a PS VR,2018-11-21T10:54:45.000Z,Comment,jBcHv3h1pHs,samsung-vr.csv,"[can, t, compete, with, a, ps, vr]","[-0.080483355, 0.07329093, 0.14160651, 0.01256...",1,0


In [22]:
out_df = comments[['Comment','CreateTimeStamp','Type', 'videoID','SENTIMENT_RF','SENTIMENT_KN']]

In [24]:
out_df.head(5)

,Comment,CreateTimeStamp,Type,videoID,SENTIMENT_RF,SENTIMENT_KN
0,I ll just leave this right here,2017-07-27T19:00:40.000Z,Comment,jBcHv3h1pHs,1,0
1,How does sound work,2018-11-30T13:18:40.000Z,Comment,jBcHv3h1pHs,0,0
2,hey,2018-11-29T04:51:42.000Z,Comment,jBcHv3h1pHs,0,0
3,Whats the song in the beginning,2018-11-23T20:46:13.000Z,Comment,jBcHv3h1pHs,0,1
4,Can t compete with a PS VR,2018-11-21T10:54:45.000Z,Comment,jBcHv3h1pHs,1,0


In [26]:
out_file_name = '../outdata/video_comments_analysis_sentiments.csv'
out_df.to_csv(out_file_name, sep='\t', index=False)

In [27]:
print('****End of script****')

****End of script****
